In [1]:
import numpy as np

import pandas as pd

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import lxml
from pandas.io.json import json_normalize

print('Libraries imported!')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported!


In [27]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki = requests.get(url).text

In [28]:
soup = BeautifulSoup(wiki, 'lxml')
type(soup)

bs4.BeautifulSoup

In [29]:
table = soup.find('table', class_='wikitable sortable')

In [30]:
table_header = table.find_all('tr')

table_head = []
for tr in table_header:
    th = tr.find_all('th')
    row1 = [tr.text.strip() for tr in th if tr.text.strip()]
    if row1:
        table_head.append(row1)


t1 = pd.DataFrame(table_head)

t1

,0,1,2
0,Postcode,Borough,Neighbourhood


In [31]:
table_rows = table.find_all('tr')

table_body = []
for tr in table_rows:
    td = tr.find_all('td')
    row2 = [tr.text.strip() for tr in td if tr.text.strip()]
    if row2:
        table_body.append(row2)

t2 = pd.DataFrame(table_body) 
t2.head()

,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [32]:
t3 = pd.concat([t1,t2])
t3.columns = t3.iloc[0]
t3.head()

,Postcode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village


In [33]:
# Drop top
t4 = t3[1:]
t4.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [34]:
# cancel rows with No Assigned Borough
t5 = t4.drop(t4[t4['Borough'] == 'Not assigned'].index)
t5

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [37]:
t5.replace('Not assigned', np.nan, inplace=True)
t5.ffill(axis=1)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [38]:
#Group Data
t5['Postcode'] = t5['Postcode'].astype(str)
t5['Borough'] = t5['Borough'].astype(str)
t5['Neighbourhood'] = t5['Neighbourhood'].astype(str)
t5.set_index(['Postcode','Borough'],inplace=True)
t5 = t5.groupby(level=['Postcode','Borough'], sort=False).agg( ','.join)

In [39]:
t5 = t5.reset_index()
type(t5)

pandas.core.frame.DataFrame

In [40]:
t5.shape

(103, 3)

In [41]:
toro = pd.read_csv('https://cocl.us/Geospatial_data')
toro.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [42]:
t5 = t5.join(toro)
t5

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Heights,Lawrence Manor",M1G,43.770992,-79.216917
4,M7A,Queen's Park,nan,M1H,43.773136,-79.239476
5,M9A,Etobicoke,Islington Avenue,M1J,43.744734,-79.239476
6,M1B,Scarborough,"Rouge,Malvern",M1K,43.727929,-79.262029
7,M3B,North York,Don Mills North,M1L,43.711112,-79.284577
8,M4B,East York,"Woodbine Gardens,Parkview Hill",M1M,43.716316,-79.239476
9,M5B,Downtown Toronto,"Ryerson,Garden District",M1N,43.692657,-79.264848


In [43]:
t5.shape

(103, 6)